<a href="https://colab.research.google.com/github/isakdiaz/treenet-colab/blob/main/treenet_coreml_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pprint
import json
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
print(tf.__version__)
print(np.__version__)

## Load Cloud Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install CoreML Tools
### Requires Python 3.5 +

In [ ]:
!pip install coremltools

In [ ]:
import coremltools as ct

## Download dataset to get classes


In [ ]:
# Dataset URL
!gdown https://drive.google.com/uc?id=1N-9k7Aa-GRAoiuKgM7ERPn-UkKdgcfIa

In [ ]:
# Place the plantnet zipped file in a folder called plantnet on your Google Drive directory
# Unzips files from google drive to google colab
!unzip /content/treenet.zip

## Create Class Labels for CoreML Config File

In [ ]:
# Get labels from folder names
DATASET_PATH = '/content/treenet'
children= [os.path.join(DATASET_PATH, child) for child in os.listdir(DATASET_PATH)]
files = filter(os.path.isdir, children)
labels = sorted([file.split("/")[-1] for file in files])
num_classes = len(labels)

print(f"Dataset contains {num_classes} classes.")
labels

# Load Model File


In [ ]:
# Check if a model exists
!ls -all -h /content/drive/MyDrive/saved_models/treenet

## Load Keras Model

In [ ]:
gdrive_dir = "/content/drive/MyDrive"
model_name = "treenet_20210929"
file_path = "{0}/saved_models/treenet/{1}.h5".format(gdrive_dir, model_name)
tf_model = tf.keras.models.load_model(file_path)

In [ ]:
tf_model.summary()

# Convert Keras model to CoreML Model


In [ ]:
# image_input = ct.ImageType(color_layout="RGB", scale=1/127.0, bias=[-1,-1,-1]) # -1 to 1 input
image_input = ct.ImageType(color_layout="RGB", scale=1/255.0, bias=[0,0,0]) # 0 to 1 input
classifier_config = ct.ClassifierConfig(labels)

# Set input as ImageType so CoreML can automatically resize it using Vision framework
coreml_model = ct.convert(tf_model, inputs=[image_input], classifier_config=classifier_config)

In [ ]:
# # Define Spec Function
from coremltools.models.neural_network.builder import _get_nn_spec as get_nn

## Get Spec and check preprocessing
spec = coreml_model.get_spec()
nn = get_nn(spec)
print(nn.preprocessing)

## Save CoreML Model to Google Drive

In [ ]:
# !ls -all -hs "$gdrive_dir"
folder_name = gdrive_dir + "/CoreML"
!mkdir -p {folder_name}
coreml_file_path = "{0}/{1}.mlmodel".format(folder_name, model_name)
coreml_model.save(coreml_file_path)
print("Core ML model {} saved in {}".format(model_name, folder_name))